In [10]:
import pandas as pd
import numpy as np

# -------------------------------
# 1) Sample data setup (example)
# -------------------------------
data = pd.read_csv('../Data/train_data.csv')
df = pd.DataFrame(data)
df['date'] = pd.to_datetime(df['date'])

print("Original DataFrame:")
print(df)
print("")

# ------------------------------------------------
# 2) Create a daily grouping and calculate IQR
# ------------------------------------------------
# Extract just the calendar day (YYYY-MM-DD) from the datetime
# Note: Using day of year might be better to handle seasonality across years
# df['day_of_year'] = df['date'].dt.dayofyear
# For simplicity, using day of month as in the original example
df['day'] = df['date'].dt.day

# Calculate quartiles and IQR for each day
grouped = df.groupby('day')['mean']
stats = grouped.quantile([0.25, 0.75]).unstack(level=1)
stats['IQR'] = stats[0.75] - stats[0.25]
stats['lower_bound'] = stats[0.25] - 1 * stats['IQR']
stats['upper_bound'] = stats[0.75] + 1 * stats['IQR']

# Merge the outlier bounds back onto the original DataFrame
df = df.merge(stats[['lower_bound', 'upper_bound']], left_on='day', right_index=True)

# Identify outliers
df['is_outlier'] = (
    (df['mean'] < df['lower_bound']) |
    (df['mean'] > df['upper_bound'])
)

# ------------------------------------------------
# 3) Compute mean of non-outliers for each day
# ------------------------------------------------
# Calculate the mean for each day, excluding the identified outliers
daily_non_outlier_mean = df[~df['is_outlier']].groupby('day')['mean'].mean()
daily_non_outlier_mean.name = 'daily_non_outlier_mean'

# Merge this back to original df on 'day'
df = df.merge(daily_non_outlier_mean, left_on='day', right_index=True)

# ------------------------------------------------
# 4) Replace outliers with the daily non-outlier mean
# ------------------------------------------------
df.loc[df['is_outlier'], 'mean'] = df.loc[df['is_outlier'], 'daily_non_outlier_mean']

print("DataFrame with outliers replaced by daily non-outlier mean:")
print(df)

#print number of outliers replaced
num_outliers_replaced = df['is_outlier'].sum()
print(f"Number of outliers replaced: {num_outliers_replaced}")
# ------------------------------------------------
# 5) (Optional) Clean up columns
# ------------------------------------------------
df.drop(columns=['day','lower_bound','upper_bound','is_outlier','daily_non_outlier_mean'], inplace=True)

print("")
print("Final Cleaned DataFrame:")
print(df)

# Save the cleaned DataFrame to a new CSV file
df.to_csv('../Data/cleaned_train_data_mean_replace.csv', index=False)

Original DataFrame:
            date   mean    max   min
0     1933-01-01  13.67  18.89  2.78
1     1933-01-02  13.89  19.61  1.89
2     1933-01-03  10.83  16.28  0.78
3     1933-01-04  14.94  21.28  1.28
4     1933-01-05  18.11  23.50  6.89
...          ...    ...    ...   ...
33281 2024-03-27  12.67  21.28  5.61
33282 2024-03-28  12.33  20.11  4.22
33283 2024-03-29  12.11  16.78  5.50
33284 2024-03-30   9.17  14.61  7.50
33285 2024-03-31   8.67  11.00  7.61

[33286 rows x 4 columns]

DataFrame with outliers replaced by daily non-outlier mean:
            date   mean    max   min  day  lower_bound  upper_bound  \
0     1933-01-01  13.67  18.89  2.78    1       1.9400       31.940   
1     1933-01-02  13.89  19.61  1.89    2       2.2300       31.720   
2     1933-01-03  10.83  16.28  0.78    3       2.1800       31.820   
3     1933-01-04  14.94  21.28  1.28    4       2.2200       31.560   
4     1933-01-05  18.11  23.50  6.89    5       2.1200       31.940   
...          ...    ...